# Code The Backpropagation


Esta implementação foi baseada no exemplo do notebook backpropagation passo-a-passo, tanto os dados quanto as ideias.

In [1]:
import numpy as np

In [40]:
class Neuron:
    def __init__(self, weights, bias):
        self.weights = weights
        self.bias = bias
        
    def calculate_output(self, inputs):
        self.output = self.activation(self.net(inputs))
        return self.output
        
    # Realiza o calculo de soma(W*x + b)    
    def net(self, inputs):
        return np.dot(self.weights, inputs) + self.bias    
    
    # Função de ativação, no caso estamos utilizando a função logistica f(x) = 1/(1 + exp(-x)) que gera curva sigmoide
    def activation(self, net):
        return 1.0 / (1.0 + np.exp(-net))

    # Derivada da função logistica, se f'(x) = f(x)(1 - f(x))
    def derivative_activation(self, out):
        return out*(1 - out)
        

In [67]:
class NeuralNetwork:
    LEARNING_RATE = 0.5
    
    # Número de entradas de dados, número de nós na camada oculta e número de nós de saida.
    def __init__(self, num_inputs, num_hidden, num_outputs):
        self.num_inputs = num_inputs
        self.num_hidden = num_hidden
        self.num_outputs = num_outputs
        
        # OBS: Dados de teste do artigo que serviu de base
        self.hidden_layer = [ Neuron([.15, .20], .35), Neuron([.25, .30], .35) ]
        self.output_layer = [ Neuron([.40, .45], .60), Neuron([.50, .55], .60) ]
        
        # self.hidden_layer = self.init_layer(num_inputs, num_hidden)
        # self.output_layer = self.init_layer(num_hidden, num_outputs)
        
    # Inicia as camadas de neuronios com pesos gerados aleatoriamente para cada neuronio     
    def init_layer_weights(self, num_weights, num_neurons):
        neurons_layer = []
        for i in range(0, num_neurons):
            weights = np.random.randn(num_weights) * 0.05 
            bias = (np.random.randn(1) * 0.05)[0] 
            neurons_layer.append(Neuron(weights, bias))
        return neurons_layer
        
    def pass_forward(self, inputs):
        outputs_hidden = self.feed_forward(inputs, self.hidden_layer)
        return self.feed_forward(outputs_hidden, self.output_layer)
        
    def feed_forward(self, inputs, neurons):
        outputs = []
        for neuron in neurons:
            outputs.append(neuron.calculate_output(inputs))
        return outputs
    
    def train(self, inputs, targets):
        """
            TODO:
            pseudo
            initialize network weights (often small random values)
              do
                 foreach training example, x
                    prediction = neural-net-output(network, x)  // forward pass
                    actual = teacher-output(x)
                    compute error (prediction - actual) at the output units
                    compute deltas for all weights from hidden layer to output layer  // backward pass
                    compute deltas for all weights from input layer to hidden layer   // backward pass continued
                    update network weights   // input layer not modified by error estimate
              until all examples classified correctly or another stopping criterion satisfied
              return the network
        """
        pass
    
    def predict(self, input):
        """
            TODO: 
        """
        pass
    
    def calculate_error(self, target, output):
        return (0.5)*(target - output)**2
    
    def calculate_total_error(self, targets, outputs):
        total_error = 0.0
        for target, output in zip(targets, outputs):
            total_error += self.calculate_error(target, output)
        return total_error
    
    def print_output_weights(self):
        print(self.output_layer[0].we)
    

Para testar, utilizaremos os seguintes dados do exemplo.

In [68]:
inputs = [.05, .10]
outputs = [.01, .99]

nn = NeuralNetwork(num_inputs=2, num_hidden=2, num_outputs=2)
nn_out = nn.pass_forward(inputs)
nn_out

[0.7513650695523157, 0.7729284653214625]

In [64]:
nn.calculate_total_error(nn_out, outputs)

0.2983711087600027

Apos o backward pass os pesos devem ser alterados para os seguintes valores:
[0.35891648, 0.408666186], [ 0.511301270, 0.561370121]

In [75]:
for n in nn.output_layer:
    print(n.weights)

[0.4, 0.45]
[0.5, 0.55]


In [36]:
# import a dataset
from sklearn import datasets
iris = datasets.load_iris()

X = iris.data
y = iris.target

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .5)

nn = NeuralNetwork(num_inputs=4, num_hidden=2, num_outputs=3)
nn_out = nn.pass_forward(X_train[0])
print(nn_out)
print(nn.calculate_total_error(nn_out, outputs))

[0.5103727736102812, 0.48429897690471635, 0.49660195034821436]
0.25305321866503117
